IPython notebook created by BIOL3014/BINF7000 course staff. Material is derived from 2014-? Practical 2.

Bug fixes: m.stantoncook@uq.edu.au, m.boden@uq.edu.au

Source: https://github.com/UQ-BIOL3014/Practical2

---

# BIOL3014/BINF700 Practical 2

## Probabilistic motif discovery
---

* **Due:** 11AM 12/08/2015
* **Revision:** 1
* **Marks:** 
    * **BIOL3014** - 8 marks. 
    * **BINF7000** - 12 marks.
---


### Objectives 

Below are a number of exercises that aim to guide you through issues and help you understand concepts related primarily to:
* Motif discovery
* Gene ontology
* Statistical testing (BINF7000)

---


### Submission requirements

Please export this IPython notebook (with written answers & completed code) to `STUDENTNUMBER_P2.ipynb` notebook and submit it via the BlackBoard portal.

----


### Resources

#### Python resources:
* The UQ Bioinformatics Python Guide (on Blackboard)
* The [python 2 documentation]. For those unfamiliar with Python the [official tutorial] is recommended
* The software carpentry [novice python lessons]

[python 2 documentation]: https://docs.python.org/2/
[offical tutorial]: https://docs.python.org/2/tutorial/index.html
[novice python lessons]: http://swcarpentry.github.io/python-novice-inflammation/

#### Other:
* [Markdown cheatsheet]

[Markdown cheatsheet]: https://github.com/adam-p/markdown-here/wiki/Markdown-Here-Cheatsheet#links

---

## Part 1 ##
The target workload for Part 1 corresponds roughly to two hours of practical work. This work should ideally be completed within the next scheduled computer lab session (supported by a tutor or lecturer), and two hours of additional work that should be completed during unscheduled time. Part 2 and 3 are for BINF7000 students only.

We will be using the custom `uqbinfpy` Python modules for this practical. The `uqbinfpy` library is available at [GitHub]. Data files are available on Blackboard under the "Practicals/workshops" tab, as well as at [GitHub].

[GitHub]: https://github.com/UQ-BIOL3014/

You should have installed the uqbinfpy library during Practical 1.

Import the following modules into Python:


```python
   from sequence import *
   from prob import *
   from symbol import *
   from webservice import *
```

### Exercise 1 (1 marks) ###

Create a “background distribution” of amino acids that is suited for scoring motifs in human protein sequences. First, consider the background used for constructing BLOSUM62 (a popular substitution matrix) by downloading and viewing `blosum62.distrib`. This file can be read using the method `readDistrib` in `prob.py`. 

```python
   bg = readDistrib('blosum62.distrib')
```

Second, construct your own from the human protein reference database (HPRD; a copy in FASTA format `HPRD.fa` is available with UniProt identifiers within the course Python code). 

Compare the backgrounds, and report if the same amino acids are the most probable and the least probable. Enter your written response into the cell labelled *Exercise 1.* This is a markdown cell. For formatting options see the [Markdown cheatsheet].

[Markdown cheatsheet]: https://github.com/adam-p/markdown-here/wiki/Markdown-Here-Cheatsheet

#### Tips: #### 
* An instance of the class `Distrib` can be inspected like so:

```python
   print bg # refers to __str__ defined for Distrib in prob.py
```

* Save the background distribution from HPRD in a file named `HPRD.distrib` using the `writeDistrib` method:

```python
   bg.writeDistrib('HPRD.distrib')
```

*Exercise 1. *

---

Helix-turn-helix (HTH) is a 20+ residue long DNA binding signature found in many transcription factors - proteins that regulate gene expression. An example structure is depicted on the right. See more at http://en.wikipedia.org/wiki/Helix-turn-helix.


## Exercise 2 ( 1 marks) ##

`hth_40.fa` contains 40 human HTH proteins. Look one up in Uniprot to see where the domain is located. Provide the accession identifier and sequence in the Markdown cell labelled *Exercise 2* and mark the HTH domain. 

**Note**: not all exemplars will be annotated with domains on Uniprot, but some will.

*Exercise 2*

---

## Exercise 3 ( 2 marks) ##

Run motif discovery by Gibbs sampling on this data a few times. The code below is a simple example of how the Gibbs sampler can be used:


```python
   from gibbs import *

   seqs = readFastaFile('hth_40.fa', Protein_Alphabet)
   W = 24 # the width of the motif sought
   g = GibbsMotif(seqs, W)
   q = g.discover()
   p = g.getBackground()
   a = getAlignment(seqs, q, p)
   k = 0
   for seq in seqs:
      print "%s \t%s" % (seq.name, seq[a[k]:a[k]+W])
      k += 1
```

### Part 1)

Copy the above code and add comments explaining what the variables `seqs`, `g`, `q`, `p` and `a` represent (as an example, a comment has already been added to the variable `W`).

Enter your code into the cell labelled *Exercise 3, part 1*.


### Part 2)

Try at least three times and also with three different window sizes (`W`). Compare the outcomes of different runs. You should do this by visually comparing motifs using [WebLogo], which takes an alignment like that printed in the code example above. 

[WebLogo]: http://weblogo.berkeley.edu

Inspect and report the potential overlap between the three motifs found. Enter your written response into the markdown cell labelled *Exercise 3, part 2*. 

Select one of your motifs (the one with the highest log-odd score) and save it to a file:

```python
   writeDistribs(q, 'hth_q.distrib')
   p.writeDistrib('hth_p.distrib')
```

Note that we use two files: one for the foreground and one for the background. Look in the files to understand the format (foreground has many distributions - one for each position; background has only one).


In [2]:
# Exercise 3, part 1

*Exercise 3, part 2* 

---

## Exercise 4  ( 4 marks) ##

You are now going to construct position specific scoring matrices (log-odd matrices), otherwise also called “position weight matrices” (PWMs). Construct PWMs from the probabilities defining your motif, using different definitions of background (you constructed two above, then the background in Gibbs is based on the training sequences excluding the motifs). The class `PWM` is defined in `sequence.py` module.

```python
   pwm = PWM(q, p)
```

### Part 1) ###
Use your PWMs to search for occurrences in the human protein set. Pick a few in your list of predictions, make sure they do not already occur in your “discovery set”, and look in their respective Uniprot entries. Are they likely to have a HTH domain? Enter your response into the markdown cell labelled *Exercise 4, part 1*.

### Part 2) ###

Investigate the GO terms of all your predictions as exemplified below. Note that the example only looks at the first hit in sequences with one or more matches, whereas you need to investigate all your predictions. Enter your code into the cell labelled *Exercise 4, part 2*.

```python
   ids = []
   seqs = readFastaFile('HPRD.fa', Protein_Alphabet)
   for seq in seqs:
      hits = pwm.search(seq)
      if len(hits) > 0:
         print "%s \t%d \t%s \t%5.3f" % (seq.name, hits[0][0], hits[0][1], hits[0][2])
         ids.append(seq.name)
   report = getGOReport(ids) 
   for row in report:
      print "%s \t%d \t%s" % row
```

### Part 3) ###
What is the function of the discovered motif according to your analysis? Use examples from your GO term analysis to support your argument. Enter your response into the markdown cell labelled *Exercise 4, part 3*.


*Exercise 4, part 1*

In [3]:
# Exercise 4, part 2

*Exercise 4, part 3*

---

## BINF7000 students ##

## Part 2 ##

The following exercises are "theoretical questions" that should be researched by perusing the textbook and relevant literature and should take about three hours outside of class time to answer.

---

## Excerise 5 ( ? marks) ##

You are given a set of sequences *S*<sub>1</sub>, *S*<sub>2</sub>, ..., *S*<sub>n</sub> and wish to identify a sequence motif that occurs at an unknown start position *a*<sub>1</sub>, *a*<sub>2</sub>, ..., *a*<sub>n</sub>  exactly once, in each sequence.

Sequence profiles or so-called position weight matrices (PWMs) are usually based on
* observed probabilities of residues in specific positions: *q*(*r*, *i*) is the probability of residue *r* in position *i*, and 
* observed prior probabilities of residues: *p*(*r*) is the probability of residue *r*, at any position.

We define an element of a PWM as *W*(*r*, *i*) = log(*q*(*r*, *i*)/*p*(*r*)).

With reference to this definition and above notation, describe the two steps of discovering motifs in protein sequences using “Gibbs sampling”. 

Now, copy the code in the `discover` method in `gibbs.py` and add Python comments about these steps, demonstrating that you understand the discovery process in terms of the code. The code has already a great number of annotations to guide you, but you have to add comments regarding the establishment and modification of the distribution and of the PWM (for example, what is the reason of the loop starting in line 60? And at 84 and 92? And at 104, 114 and 119? And the if statement at line 126?). Enter your commented code into the cell labelled *Exercise 5*. Start your comments with three hashes ### to distinguish them from the ones already there.



In [4]:
# Exercise 5

---

## Exercise 6 ( ? marks) ##

### Part 1 ###

In the Excerise 3 in Part 1, where you used Gibbs sampling to find motifs, the method found different motifs, with different likelihood scores, with the same width. Explain how this is possible in the markdown cell labelled *Exercise 6, part 1*. Refer to the specific step(s) in the code by using the comments you inserted in Exercise 5.

### Part 2 ###

You may have noticed that many results from the same data set (even when the same width is maintained) overlap with one another - representing “shifted” versions of essentially the same motif. Explain the tendency of found motifs to differ by one or more positions and enter your response into the markdown cell labelled *Exercise 6, part 2*.

*Exercise 6, part 1*

*Exercise 6, part 2*



---

## Exercise 7 ( ? marks) ##

Consider the following data set: 


| Gene | Label | Score |
| ---- | ----- | ----- |
| P123 | Yes   | 3.4   |
| P124 | Yes   | 2.4   |
| P129 | No    | 2.9   |
| P222 | No    | 3.5   |
| P223 | Yes   | 1.2   |
| P323 | Yes   | 2.1   |
| P321 | Yes   | 1.0   |
| P333 | Yes   | 1.4   |
| P334 | No    | 2.2   |
| P339 | No    | 4.0   |


If you require “positives” to have a score of 2.3 or above, you can investigate if this class is statistically enriched with the label “No” using the “Fisher’s exact test”. It is a statistical test often used to investigate the enrichment of GO terms assigned to a set of “positives” relative a background set (“positives” + “negatives”). The test allows you to determine the *p*-value for an event (the probability of the event or an event more extreme).

Study the [Fisher's exact test]. For the problem above: What p-value can you assign to the class of “positives”? Why is the Chi-squared test not suitable for this particular data set? Enter your answers into the markdown cell labelled *Exercise 7*.

**Tip**: There is an implementation in Python `getFETpval` in `stats.py`. 

[Fisher's exact test]: http://en.wikipedia.org/wiki/Fishers_exact_test




*Exercise 7*

---

## Part 3 ##

Part 3 (“implementation” and “analysis”) is expected to vary in time commitment depending on target mark (0-3 marks; about the same in terms of hours).

---

## Excerise 8 ##

As discussed in the original paper by Lawrence et al (1993), Gibbs sampling for motif discovery can be modified in ways that may improve its performance (for instance the issue mentioned in Exercise 6, part 1). Explore ways of improving the performance on the HTH data set. You need to modify or replace the current Python implementation. You need to evaluate the difference in performance (accuracy and/or efficiency) by comparing to the original implementation.

### Part 1) ###
Enter your modified implementation of Gibbs sampling for motif discovery in the cell labelled *Exercise 8, part 1*.

### Part 2) ###
Enter your evaluation of the performance of your modifications in the markdown cell labelled *Exercise 8, part 2*.


In [5]:
# Exercise 8, part 1

*Exercise 8, part 2*

---

## Exercise 9 ##

Choose a different family of proteins and collect sequence data, from which you then discover and functionally characterise motifs using the original method. For example:
1. Go to [PROSITE], browse “documentation entries” and select a domain that you think is interesting.
2. Retrieve sequence entries that match the PROSITE domain (“Retrieve a list of all…”). Further filter this set to contain sequences with less than 50% sequence similarity and possibly be from a single organism. (There are options to do this in Uniprot. Make sure you have a reasonable number of sequences to work with, e.g. more than 25.)
3. Run Gibbs motif discovery on the sequences and report on found patterns and their potential function. Compare to patterns that are documented in the PROSITE entry.
[PROSITE]: http://prosite.expasy.org/

Enter your response into the markdown cell labelled *Exercise 9*.

*Exercise 9*


---

## Exercise 10 ##

Exercise 4 resulted in a long list of GO terms that were associated with at least one of a special set of proteins. Since this type of analysis does not take into account what terms that occur more often “by chance” we are unable to say which are statistically “significant.” 

### Part 1) ###

Solve the following example problem using Fisher’s exact test: 
62 genes in the entire human genome are annotated with GO:0002253 “Activation of immune response”. 23508 genes are not. Assume now that you discover a particular property that applies to 95 of all human gene products. Five of them are annotated with GO:0002253 and (thus) 90 are not. In trying to shed light on the potential role of your discovery, you need to establish if this 5 vs. 90 and 62 vs. 23508 correlation is significant. To do this, consider and use the method `getFETpval` in `stats.py`. Enter your response in the markdown cell labelled *Exercise 10, part 1*.

### Part 2 ###
The previous exercise shows that Fisher’s exact test can assign a *p*-value to each term on basis of the number of “positive” gene products that has the term, and that does not have the term relative the total number of (relevant) gene products that has and does not have the term. In fact, `getGOReport` in `webservice.py` performs this test when the user also supplies a background data set.

Copy the code into the cell labelled *Exercise 10, part 2*. Comment the code thoroughly, demonstrating that you understand the method of determining a *p*-value for each GO term that is associated with at least one sequence in the positive set.

*Exercise 10, part 1*

In [6]:
# Exercise 10, part 2

---